### Chapter 11
## Model Evaluation

### 11.0 Introduction
### 11.1 Cross-Validating Models
#### Problem
You want to evaluate how well your model will work in the real world

#### Solution
Create a pipeline that preprocesses the data, trains the model, and then evaluates it using cross-validation:

In [1]:
# load libraries
from sklearn import datasets, metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# load digits dataset
digits = datasets.load_digits()

# create features matrix
features = digits.data

# create target vector
target = digits.target

# create standardizer
standardizer = StandardScaler()

# create logitic regression object
logit = LogisticRegression()

# create a pipeline that standardizes, then runs logistic regression
pipeline = make_pipeline(standardizer, logit)

# create k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=1)

# conduct k-fold cross-validation
cv_results = cross_val_score(pipeline, # Pipeline
                             features, # feature matrix
                             target, # target vector
                             cv=kf, # cross-validation technique,
                             scoring="accuracy", # loss function
                             n_jobs=-1) # use all CPU cores

# calculate mean
cv_results.mean()

0.964931719428926

#### Discussion
#### See Also
* Why every statistician should know about cross-validation (https://robjhyndman.com/hyndsight/crossvalidation/)
* Cross-Validation Gone Wrong (http://betatim.github.io/posts/cross-validation-gone-wrong/)

### 11.2 Creating a Baseline Regression Model
#### Problem
You want a simple baseline regression model to compare against your model
#### Solution
Use scikit-learn's DummyRegressor to create a simple model to use as a baseline:

In [2]:
# load libraries
from sklearn.datasets import load_boston
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split

# load data
boston = load_boston()

# create features
features, target = boston.data, boston.target

# make test and training split
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=0)

# create a dummy regressor
dummy = DummyRegressor(strategy='mean')

# "Train" dummy regressor
dummy.fit(features_train, target_train)

# Get R-squared score
dummy.score(features_test, target_test)

-0.001119359203955339

To compare, we train our model and evaluate the performance score:

In [4]:
# load library
from sklearn.linear_model import LinearRegression

# train simple linear regression model
ols = LinearRegression()
ols.fit(features_train, target_train)

# get R-squared score
ols.score(features_test, target_test)

0.6353620786674623

#### Discussion
DummyRegressor allows us to create a very simple model that we can use as abaseline to compare against our actual model. This can often be useful to simulate a "naive" existing prediction process in a product or system. For example, a product might have been originally hardcoded to assume that all new users will spend $100 in the first month, regardless of their features. If we encode that assumption into a baseline model, we are able to concretely state the benefits of using a machine learning approach.

DummyRegressor uses the strategy parameter to set the method of making predictions, including the mean or median value in the training set. Furthermore, if we set strategy to constant and use the constant parameter, we can set the dummy regressor to predict some constant value for every observation:

In [5]:
# create dummy regressor that predicts 20's for everything
clf = DummyRegressor(strategy='constant', constant=20)
clf.fit(features_train, target_train)

# evaluate score
clf.score(features_test, target_test)

-0.06510502029325727

One small note regarding score. By default, score returns the coefficient of determination (R-squared, $R^2$) score:

$$R^2 = 1 - \frac{\sum(y_i - \hat y_i)^2}{\sum(y_i - \bar y)^2}$$

where $y_i$ is the true value of the target observation, $\hat y_i$ is the predicted value, and $\bar y$ is the mean value for the target vector

The closer $R^2$ is to 1, the more of the variance in the target vector that is explained by the features.

### 11.3 Creating a Baseline Classification Model

#### Problem
You want a simple baseline classifier to compare against your model

#### Solution
Use scikit-learn's DummyClassifer

In [1]:
# load libraries
from sklearn.datasets import load_iris
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split

# load data
iris = load_iris()

# create target vector and feature matrix
features, target = iris.data, iris.target

# split into training and test set
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=0)

# create dummy classifier
dummy = DummyClassifier(strategy='uniform', random_state=1)

# "train" model
dummy.fit(features_train, target_train)

# get accuracy score
dummy.score(features_test, target_test)

0.42105263157894735

#### Discussion
#### See Also
scikit-learn documentation: DummyClassifier (http://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html)

### 11.4 Evaluating Binary Classifier Predictions
#### Problem
Given a trained classification model, you want to evaluate it's quality

#### Solution
Use scikit-learn's cross_val_score to conduct cross-validation while using the `scoring` parameter to define one of a number of performance metrics, including accuracy, precision, recall, and $F_1$

Accuracy is a common performance metric. It is simply the proportion of observations predicted corrected

$$Accuracy = \frac{TP + TN}{TP + TN + FP + FN}$$

where:
* $TP$ is the number of true positives. Observations that are part of the positive class (has the disease, pruchased the product, etc) and that we predicted correctly.

* $TN$ is the number of true negatives. Observations that are part of the negative class (does not have the disease, did not purchase hte product, etc.) and that we predicted correctly.

* $FP$ is the number of false positives. Also called Type 1 error. Observations predicted to be part of hte positive class that are actually part of the negative class.

* $FN$ is the number of false negatives. Also called Type 2 error. Observations predicted to be part of the negative class that are actually part of the positive class.

We can measure accuracy in three-fold (the default number of folds) cross-validation by setting `scoring='accuracy'`

In [3]:
# load libraries
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification

# generate features matrix and target vector
X, y = make_classification(n_samples = 10000,
                           n_features = 3,
                           n_informative = 3,
                           n_redundant = 0,
                           n_classes = 2,
                           random_state = 1)

# create logistic regression
logit = LogisticRegression()

# cross-validate model using accuracy
cross_val_score(logit, X, y, scoring='accuracy')

array([0.95170966, 0.9580084 , 0.95558223])

Precision is the proportion of every observation predicted to be positive that is actually positive. We can think about it as a measurement of noise in our predictions-- that is, when we predict something is positive, how likely we are to be right. Models with high precision are pessimistic in that they only predict an observation is of the positive class when they are very certain about it. Formally, precision is:

$$Precision = \frac{TP}{TP + FP}$$

In [4]:
# cross-validate model using precision
cross_val_score(logit, X, y, scoring='precision')

array([0.95252404, 0.96583282, 0.95558223])

Recall is the proportion of every positive observation that is truly positive. Recall measures the model's ability to identify an observation of the positive class. Models with high recall are optimistic in that they have a low bar for predicting that an observation is in the positive class:

$$ Recall = \frac{TP}{TP + FN}$$

In [5]:
# cross-validate model using recall
cross_val_score(logit, X, y, scoring='recall')

array([0.95080984, 0.94961008, 0.95558223])

The $F_1$ score is the harmonic mean (a kind of average used for ratios):
$$ F_1 = 2 * \frac{Precision * Recall}{Precision + Recall}$$

#### Discussion
#### See Also
* Accuracy paradox (https://en.wikipedia.org/wiki/Accuracy_paradox)